In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.optimize import linear_sum_assignment

In [3]:
# kalman filter
from filterpy.kalman import KalmanFilter

class BoxKalman:
    def __init__(self, bbox, track_id):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([
            [1,0,0,0,1,0,0],
            [0,1,0,0,0,1,0],
            [0,0,1,0,0,0,1],
            [0,0,0,1,0,0,0],
            [0,0,0,0,1,0,0],
            [0,0,0,0,0,1,0],
            [0,0,0,0,0,0,1]
        ])

        self.kf.H = np.array([
            [1,0,0,0,0,0,0],
            [0,1,0,0,0,0,0],
            [0,0,1,0,0,0,0],
            [0,0,0,1,0,0,0]
        ])

        self.kf.P *= 10
        self.kf.R *= 1
        self.kf.Q *= 0.01

        self.kf.x[:4] = np.array(bbox).reshape(4,1)
        self.id = track_id
        self.hits = 1
        self.no_updates = 0

    def predict(self):
        self.kf.predict()
        return self.kf.x[:4].reshape(-1)

    def update(self, bbox):
        self.kf.update(bbox)
        self.hits += 1
        self.no_updates = 0

    def get_state(self):
        return self.kf.x[:4].reshape(-1)

In [7]:
# iou for matching
def iou(bb1, bb2):
    x1, y1, x2, y2 = bb1
    a1, b1, a2, b2 = bb2

    inter_x1 = max(x1, a1)
    inter_y1 = max(y1, b1)
    inter_x2 = min(x2, a2)
    inter_y2 = min(y2, b2)

    inter_w = max(0, inter_x2 - inter_x1)
    inter_h = max(0, inter_y2 - inter_y1)
    inter_area = inter_w * inter_h

    bb1_area = (x2 - x1) * (y2 - y1)
    bb2_area = (a2 - a1) * (b2 - b1)

    return inter_area / (bb1_area + bb2_area - inter_area + 1e-6)

In [5]:
# hungarian and track management
class Tracker:
    def __init__(self, max_age=20, min_hits=2, iou_th=0.2):
        self.tracks = []
        self.next_id = 0
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_th = iou_th

    def update(self, dets):
        N = len(self.tracks)
        M = len(dets)

        if N == 0:
            for d in dets:
                self.tracks.append(BoxKalman(d, self.next_id))
                self.next_id += 1
            return self.tracks

        preds = [t.predict() for t in self.tracks]
        cost = np.zeros((N,M))

        for i,p in enumerate(preds):
            for j,d in enumerate(dets):
                cost[i,j] = 1 - iou(p, d)

        row_ind, col_ind = linear_sum_assignment(cost)

        assigned_tracks = set()
        assigned_dets = set()

        for r,c in zip(row_ind, col_ind):
            if 1 - cost[r,c] >= self.iou_th:
                self.tracks[r].update(dets[c])
                assigned_tracks.add(r)
                assigned_dets.add(c)

        for j, d in enumerate(dets):
            if j not in assigned_dets:
                self.tracks.append(BoxKalman(d, self.next_id))
                self.next_id += 1

        alive_tracks = []
        for i,t in enumerate(self.tracks):
            if i not in assigned_tracks:
                t.no_updates += 1
            if t.no_updates <= self.max_age:
                alive_tracks.append(t)

        self.tracks = alive_tracks
        return self.tracks

In [6]:
# run YOLO and update/save tracks
model = YOLO("yolo11s.pt")
tracker = Tracker()

cap = cv2.VideoCapture("Football match.mp4")
out = cv2.VideoWriter(
    "Football match tracked.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    30,
    (int(cap.get(3)), int(cap.get(4)))
)

TRACK_CLASSES = {0:"person", 32:"sports ball"}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    res = model(frame, verbose=False)[0]

    dets = []
    for box in res.boxes:
        c = int(box.cls)
        if c not in TRACK_CLASSES:
            continue

        x1,y1,x2,y2 = box.xyxy[0].cpu().numpy()
        dets.append([x1,y1,x2,y2])

    tracks = tracker.update(dets)

    for t in tracks:
        if t.hits >= tracker.min_hits:
            x1,y1,x2,y2 = t.get_state()
            cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0),2)
            cv2.putText(frame,f"ID:{t.id}",(int(x1),int(y1)-5),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)

    out.write(frame)

cap.release()
out.release()

[W1103 03:36:36.972460817 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:36.986935609 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:37.510979276 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:37.512113651 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:37.943872526 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:37.945235109 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:38.353970109 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:38.355119151 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:38.777323485 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1103 03:36:38.778386651 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.


This tracking system combines YOLO detections with Kalman filtering and Hungarian matching to track players and the ball across frames. YOLO first detects objects in each frame, and I filter for only persons (class 0) and sports balls (class 32). These detections then get matched to existing tracks so each object keeps the same ID over time.

Each track uses a 7-dimensional state vector with the bounding box corners $(x_1, y_1, x_2, y_2)$ plus velocity terms. The Kalman filter handles prediction and correction in two steps. First, it predicts where each track should be:

$$x_{k|k-1} = F\,x_{k-1|k-1}, \quad P_{k|k-1} = F\,P_{k-1|k-1}\,F^T + Q$$

Then when a detection comes in, it corrects the estimate using the Kalman gain:

$$K_k = P_{k|k-1}\,H^T\,(H\,P_{k|k-1}\,H^T + R)^{-1}$$

$$x_{k|k} = x_{k|k-1} + K_k\,(z_k - H\,x_{k|k-1})$$

This prediction-correction loop smooths out noisy detections and helps tracks survive brief occlusions.

To match predictions with detections, I calculate IoU (Intersection-over-Union) between each predicted box and detected box, then convert it to a cost with $\text{cost} = 1 - \text{IoU}$. The Hungarian algorithm solves this assignment problem optimally, avoiding greedy matches that might pair the wrong objects.

Unmatched detections become new tracks with fresh IDs. Tracks that don't get matched are kept around for up to `max_age` frames in case the object reappears, then deleted if they stay unmatched. Only tracks with at least `min_hits` detections actually get displayed, which filters out false positives. This is basically the SORT tracking algorithm—it's simple and fast since it only uses motion and IoU, no appearance features.

Link to Excalidraw architecture drawing: https://excalidraw.com/#json=MceTgFV37dMjJoNJrs4dU,fvQ6rjrjNullQxR1Bd8zUg

Link to Football match tracked.mp4: https://drive.google.com/file/d/1RmxL2zUPYO47fWywQRhJM1yVGqZ_2Kuy/view?usp=sharing